Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [1]:
from datetime import datetime
import google.cloud.aiplatform as aip
import kfp , kfp.v2
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.experimental.custom_job import CustomTrainingJobOp
from google_cloud_pipeline_components.aiplatform import CustomContainerTrainingJobRunOp
from google_cloud_pipeline_components.aiplatform import (EndpointCreateOp, ModelDeployOp)
from google_cloud_pipeline_components.aiplatform import ModelUploadOp
from kfp.v2.components import importer_node
from kfp.v2 import compiler

## PROVIDE THE RELAVENT INPUTS

In [ ]:
# if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
#     BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
#     BUCKET_URI = "gs://" + BUCKET_NAME

In [2]:
PROJECT_ID = "sanchit909090" 
BUCKET_NAME = "sanchit909090-bucket"
BUCKET_URI = f"gs://{BUCKET_NAME}"
# Container_path = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest"
# @param {type:"string"}

#### Region

In [3]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

To avoid name collisions between users on resources created, we create a timestamp for each instance session, and append the timestamp onto the name of resources.

In [4]:
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

#### Vertex AI Pipelines constants

Setup up the following constants for Vertex AI Pipelines:

In [5]:
PIPELINE_ROOT = "{}/pipeline_root/w_c_m".format(BUCKET_URI)

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [6]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

Define custom model pipeline that uses components from `google_cloud_pipeline_components`

In [7]:
MODEL_DISPLAY_NAME = f"train_deploy{TIMESTAMP}"

@kfp.dsl.pipeline(name="train-endpoint-deploy" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest"
):

    custom_job_task = CustomContainerTrainingJobRunOp(
        display_name = "model-training",
        container_uri = 'gcr.io/sanchit909090/wcm:latest',
        staging_bucket=BUCKET_URI)
    
    
    import_unmanaged_model_task = importer_node.importer(
        artifact_uri= 'gs://sanchit909090-bucket/wcm/model', #replaced by bucket_uri
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",
            },
        },
    ).after(custom_job_task)

    model_upload_op = ModelUploadOp(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
        serving_container_image_uri = serving_container_image_uri
    )
    model_upload_op.after(import_unmanaged_model_task)

    endpoint_create_op = EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint" + TIMESTAMP,
    )

    
    ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type="n1-standard-8",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

## Compile the pipeline

Next, compile the pipeline.

In [8]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="wcm_pipeline.json".replace(" ", "_"),
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


## Run the pipeline

Next, run the pipeline.

In [9]:
DISPLAY_NAME = "WCM_" + TIMESTAMP

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="wcm_pipeline.json".replace(" ", "_"),
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run()

! rm wcm_pipeline.json

print('Pipeline Successfully Deployed')

Creating PipelineJob
PipelineJob created. Resource name: projects/455727625972/locations/us-central1/pipelineJobs/train-endpoint-deploy20220703120021-20220703120025
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/455727625972/locations/us-central1/pipelineJobs/train-endpoint-deploy20220703120021-20220703120025')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/train-endpoint-deploy20220703120021-20220703120025?project=455727625972
PipelineJob projects/455727625972/locations/us-central1/pipelineJobs/train-endpoint-deploy20220703120021-20220703120025 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/455727625972/locations/us-central1/pipelineJobs/train-endpoint-deploy20220703120021-20220703120025 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/455727625972/locations/us-central1/pipelineJobs/train-endpoint-deploy20220703120021-20220703120025 cur

In [10]:
1+2

3